In [1]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, os, math, random, sys, datetime
from datetime import date, timedelta, datetime
import pyautogui
from tqdm import tqdm

# chromedriver 자동 설치
import chromedriver_autoinstaller

from selenium.webdriver.common.action_chains import ActionChains


In [2]:
# 작업 시간과 고유 dir 등 생성
n = time.localtime()
s = '%04d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday)
s_time = time.time( )


a = datetime.now()-timedelta(days=1)
a = a.strftime('%Y-%m-%d')
b = datetime.now()-timedelta(days=14)
b = b.strftime('%Y-%m-%d')
print(a, b)

2021-05-06 2021-04-23


In [3]:
print("=" *80)
print("         YOUTUBE 크롤러입니다.")
print("=" *80)

f_dir = os.path.expanduser('~') + f'\\Desktop\\crawler-bot\\COMMUNITY\\{b}-{a}\\' # 운영체제 상관없이 바탕화면 'crawler-bot' 폴더에 작업

print("=" *80)
print("데이터 크롤링을 시작합니다.")
print("=" *80)

         YOUTUBE 크롤러입니다.
데이터 크롤링을 시작합니다.


In [4]:
# 저장될 파일 경로와 이름을 지정
# 파일을 저장할 폴더 위치를 만들고, 중복일 경우 해당 폴더를 삭제하고 새로 생성합니다.
try : 
    os.makedirs(f_dir)
except : pass
else : 
    os.chdir(f_dir)

fc_name=f_dir+'\\'+f'youtube.csv'

In [5]:
# 크롤링 데이터가 들어갈 리스트 생성
domain2 = []    # 크롤링 도메인
keyword2 = []   # 크롤링 키워드
title2 = []     # 게시물 제목
user2 = []      # 게시물 작성자
date2 = []     # 게시물 작성 시간
view2 = []     # 게시물 뷰 카운트

url2 = []      # 게시물 url
like2 = []     # 좋아요 - 싫어요
writer2 = []   # 댓글 작성자
com2 = []      # 댓글

In [6]:
# chromedriver-autoinstaller 이용
chromedriver_autoinstaller.install(cwd=True)

# HEADLESS MODE
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080') 
# options.add_argument("--disable-gpu")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
args = ["hide_console", ]

# 알림창 끄기
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# 웹사이트 접속 후 해당 메뉴로 이동
driver = webdriver.Chrome(options=options,service_args=args)
driver.maximize_window()

In [7]:
# 1. 유튜브 이동

keyword_list = ['이낙연', '이재명']

for keyword in keyword_list :
    print("keyword:", keyword)
    query_url= (f'https://www.youtube.com/results?search_query={keyword}&sp=CAI%253D')
    driver.get(query_url)
    driver.implicitly_wait(5)
    
    while True :        

        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(1)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        meta_box = soup.find_all(name='div', attrs={'id':'metadata-line'})

        date_list = []
        for box in meta_box :
            try :
                date_list.append(box.find_all('span')[1].text)
            except : #실시간 방송의 경우
                pass
        
        if "2주 전" in date_list :
            break
            
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')        
    item_box = soup.find_all(name='div', attrs={'class':'text-wrapper style-scope ytd-video-renderer'})
    
    for item in item_box :
        try :
            
            domain1 = "youtube"
            keyword1 = keyword

            date1 = item.find_all(name='span',attrs={'class':'style-scope ytd-video-meta-block'})[1].text
            title1 = item.find(name='a', attrs={'id':'video-title'})['title']
            url1 = "https://www.youtube.com/" + item.find(name='a', attrs={'id':'video-title'})['href']
            view1 = item.find_all(name='span',attrs={'class':'style-scope ytd-video-meta-block'})[0].text.replace("조회수", "").replace("회","").strip()

            if "천" in view1 :
                view1 = int(view1.replace(".","").replace("천","")) * 100

            elif "만" in view1 :
                view1 = int(view1.replace(".","").replace("만","")) * 1000


            for a in item.find(name='div', attrs={'id':'channel-info'}).find_all('a') :
                if len(a.text.replace("\n","")) > 1 :
                    user1 = a.text.replace("\n","")
                    break

            print(f'date : {date1}')
            print(f'title : {title1}')
            print(f'url : {url1}')
            print(f'view : {view1}')
            print(f'user : {user1}')
            print("\n")

            domain2.append(domain1)
            keyword2.append(keyword1)
            date2.append(date1)
            title2.append(title1)
            url2.append(url1)
            view2.append(view1)
            user2.append(user1)
        except : #실시간 방송이 있는 경우
            pass


keyword: 이낙연
date : 3시간 전
title : [사전공개] 미안해서 공개합니다^^ 이낙연 악플모음.zip (feat. 우려봇 여니)
url : https://www.youtube.com//watch?v=MEgmCggefxM
view : 903
user : 이낙연TV


date : 3시간 전
title : 긴급! 이낙연 방금 충격 선언! 대선에 대한 충격적인 조작의 힘이 밝혀졌다! 문재인이 주범! 윤석열과 오세훈이 출간 한 수사 증거! 끔찍한 진실이 밝혀졌습니다!
url : https://www.youtube.com//watch?v=lJZECNCKDnw
view : 562
user : 실시간 정치 TV


date : 4시간 전
title : 2021.05.07. 긴급! 이낙연 방금 충격 선언! 대선에 대한 충격적인 조작의 힘이 밝혀졌다! 윤석열과 오세훈이 출간 한 수사 증거!
url : https://www.youtube.com//watch?v=3Jv1U8eF-xk
view : 4800
user : ctistai


date : 스트리밍 시간: 12시간 전
title : [칸트TV] 민주당 대선주자 이재명 이낙연 정세균, 허경영 따라잡기
url : https://www.youtube.com//watch?v=Rt2K9q3sWGM
view : 265
user : 칸트TV


date : 14시간 전
title : (충격) 이재명 이낙연 쿠데타 일으켰다,군가산점 말고! 라임 펀드 터졌다! 유시민이 위험하다 구속 초읽기! 문재인 충격!
url : https://www.youtube.com//watch?v=30Tb2rvIGyU
view : 552
user : 특집 뉴스


date : 16시간 전
title : [11시 김광일 쇼] 이재명 "청년 여행비 1000만원" VS 이낙연 "전역땐 3000만원"
url : https://www.youtube.com//watch?v=1XwcGFfBlCw
view : 59000
user : 조선일보


date : 

date : 10분 전
title : 이재명계 들고 일어났다! 민주당 전면전 발발! 양정철 뛰고 있다! 문재인 충격! (진성호의 직설)
url : https://www.youtube.com//watch?v=RiwN7KLsYXg
view : 4100
user : 진성호방송


date : 16분 전
title : 이재명 지지율 62.5%!! 경기도 민심 충격 ㄷㄷㄷ - 그런데 경선 연기를 해???
url : https://www.youtube.com//watch?v=6SaWmRu1aiw
view : 3400
user : 언론 알아야 바꾼다


date : 1시간 전
title : '문재인 대통령 복심' 양정철, 이재명과 정권 재창출 모색? [아침에 이슈]
url : https://www.youtube.com//watch?v=sqAWtgRGSCA
view : 189
user : 뉴스TVCHOSUN


date : 1시간 전
title : 신문브리핑3 "친문이 띄운 대선 경선 연기론, 이재명 측 반발"외 주요기사 [아침&매일경제]
url : https://www.youtube.com//watch?v=y52kguEuOcU
view : 103
user : MBN News


date : 스트리밍 시간: 2시간 전
title : 이재명 떡잎부터 달랐다
url : https://www.youtube.com//watch?v=GNYai27IWB4
view : 81000
user : 가로세로연구소


date : 2시간 전
title : [JB TIMES] 방역 핑계로 경선 연기 주장? 이재명 죽이기 본격화되나
url : https://www.youtube.com//watch?v=i0VvvY572WI
view : 3800
user : MBC라디오


date : 2시간 전
title : 민주당 대선 후보가 이재명으로 쏠린 걸 민주당 내부만 모르는 듯.
url : https://www.youtube.com//watch?v=oXgrqEHB3Mo
view : 200
user : 유재

view : 23000
user : 진성호방송


date : 1주 전
title : 윤석열 vs 이재명, 2030 지지율 승자는?
url : https://www.youtube.com//watch?v=loXga8Isv_o
view : 67000
user : 채널A 뉴스TOP10


date : 1주 전
title : ‘이재명 형수 욕설 공개설’에…정세균 측 “명백한 허위”
url : https://www.youtube.com//watch?v=xcSXxVIi790
view : 57000
user : 채널A 뉴스TOP10


date : 1주 전
title : 전주혜, 이재명 판결 언급에 얼음된 천대엽 후보자?
url : https://www.youtube.com//watch?v=zcUpsBo9r9c
view : 11000
user : 대한민국 청아대


date : 1주 전
title : 이재명 "전 세계가 대한민국 경기도의 기본소득 정책을 주목!"
url : https://www.youtube.com//watch?v=D-FFzcOJA_U
view : 18000
user : OhmynewsTV


date : 1주 전
title : [지역민방 특별대담]토지보유세 늘려, 기본소득합시다! | 이재명 경기도지사
url : https://www.youtube.com//watch?v=hxRMQjYkMXg
view : 11000
user : JTV전주방송


date : 1주 전
title : [지역민방 특별대담]기본소득은 복지정책? 경제정책? | 이재명 경기도지사
url : https://www.youtube.com//watch?v=lA08R8UViK0
view : 598
user : JTV전주방송


date : 1주 전
title : [최병묵의 팩트] 민주당내 이재명계 의원 모임 뜬다는데
url : https://www.youtube.com//watch?v=sO_YBE54zZc
view : 5000
user : 최병묵의 FACT


date : 1주 전
title 

In [12]:
writer2 = []
com2 = []

In [13]:
# 댓글 크롤링

for url in tqdm(url2) :
    
    print("URL : ", url)

    driver.get(url)
    driver.implicitly_wait(5)
    time.sleep(3)
    element = driver.find_element_by_tag_name('body')
    element.send_keys(Keys.END)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    black_box = soup.find_all(name='yt-formatted-string', attrs={'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text'})

    for box in black_box :
        try :        
            text = box['aria-label']
            if "좋아요 " in text :
                like1 = text.replace('좋아요 ', '').replace(',','').replace('개','').strip()
            elif "싫어요 " in text :
                dislike1 = text.replace('싫어요 ', '').replace(',','').replace('개','').strip()
                break
        except :
            pass
    
    if like1 == "없음" :
        like1 = 0
    if dislike1 == "없음" :
        dislike1 = 0
        
    net_like = int(like1) - int(dislike1)
    print('net_like :', net_like)
    like2.append(net_like)

    element = driver.find_element_by_tag_name('body')
    element.send_keys(Keys.END)
    html = driver.page_source
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-20);")

    time.sleep(3)

    old_box = len(html)

    while True:

        element = driver.find_element_by_tag_name('body')
        element.send_keys(Keys.END)
        html = driver.page_source
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-3);")
        time.sleep(3)

        new_box = len(html)

        print(new_box, old_box)

        if new_box == old_box:
            break

        old_box = new_box

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    comment_box = soup.find(name='div', attrs={'id':'contents'}).find_all(name='ytd-comment-thread-renderer', attrs={'class':'style-scope ytd-item-section-renderer'})
    writers = []
    comments = []
    for comment in comment_box :
        

        try :
            writer1 = comment.find(name='a', attrs={'id':'author-text'}).text.strip()
            com1 = comment.find(name='yt-formatted-string', attrs={'id':'content-text'}).text.strip()
            writers.append(writer1)
            comments.append(com1)
        except :
            print("\n예외상황 :")
            print(comment.text.strip().replace("\n", ""))
            
    print("댓글 작성자 :", writers)
    print(f"댓글 내용 :", comments)
    print("\n")
    
    writer2.append(writers)
    com2.append(comments)

  0%|                                                                                          | 0/318 [00:00<?, ?it/s]

URL :  https://www.youtube.com//watch?v=MEgmCggefxM
net_like : 472
3831657 3630307
4039244 3831657
4241137 4039244
4443371 4241137
4647344 4443371
4858236 4647344
5072483 4858236
5073338 5072483
5073338 5073338


  0%|▎                                                                               | 1/318 [00:45<4:02:09, 45.84s/it]

댓글 작성자 : ['lee따뜻한 마음', '이루다', '김문재 된다', '이노을', '칼랑코에', 'rabbit silly', 'H K', 'noname', '김기회', '머핀블루베리', '쭈맘', '김정배', '아몬드', '깨소금', 'J B', '최혜진', 'abc페르소나', '쏭쏭쓰', '숲속의 다인데기', '김정배', '이현범', '4수수꽃다리', 'Love Moonlight', '1011라버브리', 'moon숙', '나밍밍', '냥노을', 'JEONG LEE', 'Olivia Cho', '김영자', 'young joung', '통영Story', '水파란', '찮은', '완니', '방배동휘발유', '이쿠하나', 'lena lee', '발목없을무', '체리세이지', '김정배', 'kim kobong', '수박', '시냇물', '윤지원', 'H K', '나루미루미', 'BAYAHRO KO', '구산사람', '민희네', '야호호', '블루green', '오직달님', 'gjjj ll', '조인순', '강조나단', 'citrine', '아라시와빅스', '김한웅', '굿세라', 'green forest', '푸우위니더', '커피향이좋다', '5월봄', '김효주', '조은', '돌개바람', '홍이', 'gmc', '권짱', 'Mr Pond', 'Raon Kim', 'MJ Il', '양양', 'Sunmi Yuu', 'ShinJung Choi', '완니', '김금손', 'qwew pop', '널널', '퀴달리에', 'myomyo Im', 'eaenjung jeon', '꽃바람', '완두콩', 'robin', 'flora', '만세대한민국', '고뤠', '김정화', '고흐', '별지나', "노는게적성에딱맞아'ᄉ'", '박에스텔 TV', '박연주', 'hj lee', '유Rick', 'wf Kast', 'fhc djc', 'sergioygkeke', 'Bless You', 'Y Y', '꾸엥', '스피드라온다온', '달팽이', '녹두', 'love S', '완두콩', '서

net_like : 8
3614799 3614800
3613908 3614799
3613919 3613908
3614069 3613919
3614069 3614069


...

댓글 작성자 : ['초롱수니', '박마루', '돌바위', '냥사랑', '한상세', 'hedgehog750514', '클롭의붉은제국', 'YK KIM', 'Kay Kim', '김선석', '김진두', 'moon202ga', '김uycgjd', '더막가삼촌', '유리Eurydice', '조경애', 'eunyoung kim', '정희성', '최은영', '행복한바다', '콩깍지', 's un', '난감하네', '부계정 댓글러', 'Dong-sik Park', "이니덕후 Moon's Big fan", '춘삼 승리', '깅경희', '가제트', 'V90 Cross', '박광호', '김미진', 'meaing gu', '수호자', 'kang웅석', '조경애', 'buzz lightyear', '박새롭게', '제비꽃', 'John B. Kim', 'Ray K', '무무', '김영', 'Jin Jin', '무무', '헬쥐', 'jelly kim', '오드리햇반', '4935 Jb', '이영숙', 'Jasonlee Comodosky', '전상수', 'l_초코링', '서미옥', '김맑음', '있는대로봐라', '모든게 은혜로다.희영', '박혜정', '딸기우유', '김영주', '안드로메다', '김민성', '전계화', '파스텔톤', '푸른별', '이주용', '이은숙', '영덕아제', '한주양', 'SCAR AN', 'sgw5799', 'The Rose', '바겨사', '공정', '오직', '백옥선', '김옥선', 'HC K', '홍성대', '권순애', '김민정', '홍선미', '김병곤', '우슬초', '고귀한사람', '밝은별', 'uncut125f', '구서필', '서순희', '푸르매', '유정숙', '이정금', '리아', '메주로핵융합', '정상', '이영우', '늘푸름', '김종남', '5am club', '빈실버', 'Minsu Gu', '다커리', 'sy lee', '푸르른', '無駄目人間.', '4935 Jb', '박상희', '사사세', '장승원', 'Red dragon', '공명

 26%|████████████████████▌                                                          | 83/318 [56:45<2:26:10, 37.32s/it]

댓글 작성자 : []
댓글 내용 : []


URL :  https://www.youtube.com//watch?v=y52kguEuOcU
net_like : 7
3267546 3267546


 26%|████████████████████▌                                                          | 83/318 [57:02<2:41:30, 41.24s/it]


AttributeError: 'NoneType' object has no attribute 'find_all'

In [14]:
df = pd.DataFrame()

df['domain'] = pd.Series(domain2)
df['keyword'] = pd.Series(keyword2)
df['title'] = pd.Series(title2)
df['channel'] = pd.Series(user2)
df['date'] = pd.Series(date2)
df['view'] = pd.Series(view2)
df['url'] = pd.Series(url2)
df['like'] = pd.Series(like2)
df['commentor'] = pd.Series(writer2)
df['comment'] = pd.Series(com2)

df

,domain,keyword,title,channel,date,view,url,like,commentor,comment
0,youtube,이낙연,[사전공개] 미안해서 공개합니다^^ 이낙연 악플모음.zip (feat. 우려봇 여니),이낙연TV,3시간 전,903,https://www.youtube.com//watch?v=MEgmCggefxM,471.0,"[lee따뜻한 마음, 이루다, 김문재 된다, 이노을, 칼랑코에, rabbit sil...",[악플도 읽으실 수 있는 그 당당함과 그 배려에 정말 감동입니다.\n어떤 사람은 자...
1,youtube,이낙연,긴급! 이낙연 방금 충격 선언! 대선에 대한 충격적인 조작의 힘이 밝혀졌다! 문재인...,실시간 정치 TV,3시간 전,562,https://www.youtube.com//watch?v=lJZECNCKDnw,472.0,[],[]
2,youtube,이낙연,2021.05.07. 긴급! 이낙연 방금 충격 선언! 대선에 대한 충격적인 조작의 ...,ctistai,4시간 전,4800,https://www.youtube.com//watch?v=3Jv1U8eF-xk,8.0,"[이정자, 진실은항상살어있잘챙기시고쪼아주세요미자, 김두진, 허명순, 이정자, 허명순...","[국민내흼응휜함니다, 70십년지킨대한민국악날한좌패거리들이점령하게됏는지한심한국민들망하..."
3,youtube,이낙연,"[칸트TV] 민주당 대선주자 이재명 이낙연 정세균, 허경영 따라잡기",칸트TV,스트리밍 시간: 12시간 전,265,https://www.youtube.com//watch?v=Rt2K9q3sWGM,697.0,[허동자 Huhdongja],[허경영이답이다]
4,youtube,이낙연,"(충격) 이재명 이낙연 쿠데타 일으켰다,군가산점 말고! 라임 펀드 터졌다! 유시민이...",특집 뉴스,14시간 전,552,https://www.youtube.com//watch?v=30Tb2rvIGyU,67.0,"[이정숙, 이정숙]",[이재명 재산좀 지금안팔 리면 국 민들 한테 팔아 나눠주라고 전국민...
...,...,...,...,...,...,...,...,...,...,...
313,youtube,이재명,긴급! 김부선 방금 이재명 문성근 충격 폭로! 이재명 대권 가도에 먹구름? (진성호...,진성호방송,2주 전,26000,https://www.youtube.com//watch?v=RqiZ-uuWxlI,NaN,NaN,NaN
314,youtube,이재명,김부선 VS 이재명 누가 진짜 지독한 거짓말쟁이인지 말해줄까?(3분30초~4분12초...,최국의 가짜뉴스,2주 전,71000,https://www.youtube.com//watch?v=T9sCpa5ZdXU,NaN,NaN,NaN
315,youtube,이재명,[충격단독] 디어문 이재명 제2의 혜경궁김씨,가로세로연구소,스트리밍 시간: 2주 전,37000,https://www.youtube.com//watch?v=fzi4Ak_SXP4,NaN,NaN,NaN
316,youtube,이재명,"이재명 ""윤석열, 오래 잘 준비한 듯…차기대선 1대 1 대결"" / JTBC 정치부회의",JTBC News,2주 전,31000,https://www.youtube.com//watch?v=2zLu-jHO8mk,NaN,NaN,NaN


In [ ]:
# csv 형태로 저장하기
df.to_csv(fc_name,encoding="utf-8-sig",index=False)

e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: csv 파일명 : %s " %fc_name)

driver.quit()
sys.exit("프로그램을 종료합니다.")



1.총 소요시간은 3585.7 초 입니다 
2.파일 저장 완료: csv 파일명 : C:\Users\JPARK-02\Desktop\crawler-bot\COMMUNITY\2021-04-23-2021-05-06\\youtube.csv 


SystemExit: 프로그램을 종료합니다.